# Grover Algorithm
## Introduction
Grover's algorithm is a quantum algorithm that finds with high probability the unique input to a black box function that produces a particular output value, using just O(√N) evaluations of the function, where N is the size of the function's domain. It was devised by Lov Grover in 1996.

References: https://github.com/OriginQ/QRunes/blob/master/source/chapters/algorithms/Grover_Algorithm.rst

## Importing the required libraries

In [24]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.compiler import transpile
from qiskit.visualization import circuit_drawer

## Functions
### Generate 3-qubit Oracle

In [25]:
def generate_3_qubit_oracle(target):
    qc = QuantumCircuit(3)
    if target == 0:
        qc.x([0, 1])
        qc.ccx(0, 1, 2)
        qc.x([0, 1])
    elif target == 1:
        qc.x(0)
        qc.ccx(0, 1, 2)
        qc.x(0)
    elif target == 2:
        qc.x(1)
        qc.ccx(0, 1, 2)
        qc.x(1)
    elif target == 3:
        qc.ccx(0, 1, 2)
    return qc

### Diffusion Operator

In [26]:
def diffusion_operator(qc, qubits):
    qc.h(qubits)
    qc.x(qubits)
    qc.h(qubits[-1])
    qc.mcx(list(range(len(qubits) - 1)), qubits[-1])  # Updated to mcx
    qc.h(qubits[-1])
    qc.x(qubits)
    qc.h(qubits)

### Grover Algorithm

In [27]:
def grover_algorithm(target):
    qr = QuantumRegister(3)
    cr = ClassicalRegister(2)
    qc = QuantumCircuit(qr, cr)

    # Prepare the ancilla qubit
    qc.x(qr[2])
    qc.h(qr[2])

    # Apply Hadamard gates to the working qubits
    qc.h(qr[:2])

    # Append the oracle
    oracle = generate_3_qubit_oracle(target)
    qc.compose(oracle, inplace=True)

    # Apply the diffusion operator
    diffusion_operator(qc, qr[:2])

    # Measure the working qubits
    qc.measure(qr[:2], cr[:2])

    # Updated execution process
    backend = Aer.get_backend('qasm_simulator')  # Directly using qiskit_aer
    transpiled_qc = transpile(qc, backend)  # Transpile the circuit for the backend
    job = backend.run(transpiled_qc, shots=1)  # Execute the quantum circuit
    result = job.result()
    counts = result.get_counts(qc)
    print(counts)

## Main

In [28]:
if __name__ == '__main__':
    for target in range(4):
        print("Target is " + str(target) + "!")
        print("Circuit being configured...")
        grover_algorithm(target)
        print("Circuit configuration complete for target {}.\n".format(target))

Target is 0!
Circuit being configured...
{'00': 1}
Circuit configuration complete for target 0.

Target is 1!
Circuit being configured...
{'10': 1}
Circuit configuration complete for target 1.

Target is 2!
Circuit being configured...
{'01': 1}
Circuit configuration complete for target 2.

Target is 3!
Circuit being configured...
{'11': 1}
Circuit configuration complete for target 3.
